In [ ]:
"""
RAMS query program

Created on Wed Jul 5 11:49:17 2017

@author: Tingting Huang

thuang1@iastate.edu

"""

In [ ]:
# use your own directory and filename

# Must use column name: Latitude, Longitude, routeId (in LRS format, e.g. "S001910235E" for I-235 EB)


dirpath = 'C:/Users/thuang1/Desktop/REST API/Weather/'
filename = 'sensors.csv'



In [ ]:
import pandas as pd
import numpy as np
import json
import requests
from datetime import datetime

In [ ]:
def get_locations(df):
    d = []
    for i in xrange(len(df)):
        x = {'geometry':{'y':df.iloc[i]['Latitude'], 'x':df.iloc[i]['Longitude']}, 'routeId':df.iloc[i]['routeId']}
        d.append(x)

    locations = json.dumps(d, ensure_ascii=False)

    return locations

In [ ]:
def get_measure(locations, tolerance):
       

    url = 'https://gis.iowadot.gov/ramsa/rest/services/lrs/MapServer/exts/LRSServer/networkLayers/0/geometryToMeasure?'
    data = {'f':'json','locations': locations,'tolerance':str(tolerance),'inSR':'4326'}
    response = requests.post(url, data=data)
    r=response.json()

    # extract results
    dat=[]
    for i in r['locations']:
        dat.append(i['results'][0]['measure'])
    dat=np.asarray(dat)
    
    return dat

In [ ]:
# load your data
df = pd.read_csv(dirpath+filename)

n=500

ans=[]
for i in xrange(len(df)/n+1):
    ans.append(df[i*n:i*n+n])

In [ ]:
# Main
now1=datetime.now()

final=[]
i=0

for a in ans:
    i=i+1
    locations = get_locations(a)
    
    dat = get_measure(locations, 50)
        
    a['MM']=dat
    
    final.append(a)
    
    print('Success! Progress: '+str(round(float(i)/len(ans)*100,2))+"%")

final=pd.concat(final,axis=0).reset_index(drop=True)

now2=datetime.now()

print('Runtime: '+str(now2-now1))

In [ ]:
# save to new file
final.to_csv(dirpath+'MM_Found_'+filename, header=True, index=False)